In [3]:
import torch
x = torch.zeros([2,3])
print(x)

tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [46]:
import torch

def shift_to_mask(x_torch, dtype):
    x_shape = x_torch.shape
    
    dtype_filter    = torch.arange(10) < (17 - dtype)
    mask            = torch.tensor([-(1 << i) for i in range(10)]) * dtype_filter
    offset          = torch.tensor([(256 >> (9 - i)) for i in range(10)]) * dtype_filter
    
    mask_flag = torch.zeros((10, x_shape[0], x_shape[1]), dtype=torch.int16)
    for idx in range(10):
        mask_flag[idx] = (torch.abs(x_torch) == (idx + 1))

    mask_block = torch.sum(mask_flag * mask.reshape(-1,1,1), dim=0)
    offset_block = torch.sum(mask_flag * offset.reshape(-1,1,1), dim=0)

    return mask_block, offset_block
    
x_torch = torch.tensor([[9, 7, 0, 0],
                        [2, 4, 6, 8]], dtype=torch.int16)
shift_to_mask(x_torch, 10)

tensor([[  0, -64,   0,   0],
        [ -2,  -8, -32,   0]])
tensor([[ 0, 32,  0,  0],
        [ 1,  4, 16,  0]])
0x20


(tensor([[  0, -64,   0,   0],
         [ -2,  -8, -32,   0]]),
 tensor([[ 0, 32,  0,  0],
         [ 1,  4, 16,  0]]))

In [42]:
import torch 

x = torch.arange(6).reshape(2,1,3,1)
y = torch.arange(10).reshape(1,2,1,5)

print(x * y)

# print((x * y).shape)

tensor([[[[ 0,  0,  0,  0,  0],
          [ 0,  1,  2,  3,  4],
          [ 0,  2,  4,  6,  8]],

         [[ 0,  0,  0,  0,  0],
          [ 5,  6,  7,  8,  9],
          [10, 12, 14, 16, 18]]],


        [[[ 0,  3,  6,  9, 12],
          [ 0,  4,  8, 12, 16],
          [ 0,  5, 10, 15, 20]],

         [[15, 18, 21, 24, 27],
          [20, 24, 28, 32, 36],
          [25, 30, 35, 40, 45]]]])


In [ ]:
import torch

def rolling_total(x_torch,length,dtype):
    x_shape = x_torch.shape

    rolling_flag = torch.zeros((10, x_shape[0], x_shape[1]), dtype=torch.int16)
    for idx in range(9):
        x_torch_value = (x_torch >= 2**(idx+6)) & (x_torch < 2**(idx+7)) | (x_torch < -(2**(idx+6))) & (x_torch >= -(2**(idx+7)))
        rolling_flag[idx] = ((x_torch_value) == True)  # rolling_value 가 1,-1d이면, idx=0에 저장.


In [53]:
import torch
import torch_dct as dct
a=torch.tensor([0,1,2,3])
b=a.type(torch.int16)
# c=a.int16()
print(torch.__version__)

SyntaxError: invalid syntax (2810875026.py, line 2)

In [72]:
import torch
import numpy as np
def display_per_block(x_numpy,length):
     x_numpy_shape = x_numpy.shape
     print(x_numpy.shape, x_numpy.dtype)
     # block 개수 는 length와 관련됨.
     # (length/2)*INT16 => 64B기준(INT16*32), => int64로 두고 shape 추가
     block_filter = np.zeros((length, x_numpy_shape[0]), dtype = np.int16)
     for cnt in range(length):
          block_filter[cnt] = np.arange(length) < (length - cnt)
          print("block_filter[cnt]",cnt,block_filter[cnt])
     
x_torch = torch.tensor([[9, 7, 0, 0],
                        [2, 4, 6, 8]], dtype=torch.int16)
x_numpy = x_torch.numpy()
display_per_block(x_numpy,2)

(2, 4) int16
block_filter[cnt] 0 [1 1]
block_filter[cnt] 1 [1 0]
